### Set up

#### 1. Set  up  accounts and role

In [1]:
#!pip install sagemaker==1.39.0

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [3]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-201910130520"
instance_type = "ml.p3.8xlarge" 

In [4]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [5]:
bucket = "aegovan-data"

In [6]:
#trainfile = "s3://{}/aimed/AIMedtrain.json".format(bucket)
#trainfile = "s3://{}/aimed/AIMedFull.json".format(bucket)
trainfile="s3://{}/aimed/AIMedtrain_preprocessed.json".format(bucket)

valfile="s3://{}/aimed/AIMedval_preprocessed.json".format(bucket)
# trainfile = "s3://{}/aimed/AIMedtrain_pubmedoverlap.json".format(bucket)
# valfile="s3://{}/aimed/AIMedval_pubmedoverlap.json".format(bucket)
#embeddingfile="s3://{}/embeddings/PubMed-and-PMC-w2v.bin.txt".format(bucket)
#embeddingfile="s3://{}/embeddings/bio_nlp_vec/PubMed-shuffle-win-30.bin.txt".format(bucket)
pretrained_bert="s3://{}/embeddings/bert/".format(bucket)


s3_output_path= "s3://{}/results/".format(bucket)
s3_code_path= "s3://{}/aimed_bert_code".format(bucket)

### Start training

In [7]:
pub_inputs = {
    "train" : trainfile,
    "val" :valfile,
    "PRETRAINED_BIOBERT" : pretrained_bert
}

In [8]:
BertNetworkFactoryhyperparameters = {
  #  "dataset":"PpiAimedDatasetFactory",
    "dataset":"PpiAimedDatasetPreprocessedFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps" : "4",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.001,
    "earlystoppingpatience":20
}

In [9]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [10]:
!git log -1 | head -1
!git log -1 | head -5 | tail -1

commit 43023662c11e1d76bddb3fc70c79dfc85bdc9ab5
    Add gradient accumulation


In [11]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
            #  'commit': '58a09e154935248667062a36fdae7d86b86b477c'
             }

In [12]:
hyperparameters = BertNetworkFactoryhyperparameters
inputs = pub_inputs 


In [13]:
hyperparameters

{'accumulation_steps': '4',
 'batchsize': '8',
 'dataset': 'PpiAimedDatasetPreprocessedFactory',
 'earlystoppingpatience': 20,
 'epochs': '1000',
 'learningrate': 0.001,
 'log-level': 'INFO',
 'network': 'RelationExtractorBioBertFactory',
 'trainfile': 'AIMedtrain_preprocessed.json',
 'valfile': 'AIMedval_preprocessed.json'}

In [14]:
inputs

{'PRETRAINED_BIOBERT': 's3://aegovan-data/embeddings/bert/',
 'train': 's3://aegovan-data/aimed/AIMedtrain_preprocessed.json',
 'val': 's3://aegovan-data/aimed/AIMedval_preprocessed.json'}

In [15]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     #entry_point='main_train_k_fold.py',
    entry_point='main_train_bert.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets', 'source/preprocessor', 'source/modelnetworks','source/trainpipelinesbuilders'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                    git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    code_location=s3_code_path,
                    base_job_name ="aimed-ppi-bert-acc")

In [16]:
estimator.fit(inputs)

2019-10-21 10:45:26 Starting - Starting the training job...
2019-10-21 10:45:28 Starting - Launching requested ML instances...
2019-10-21 10:46:26 Starting - Preparing the instances for training......
2019-10-21 10:47:47 Downloading - Downloading input data
2019-10-21 10:47:47 Training - Downloading the training image..........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-10-21 10:49:47,834 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-10-21 10:49:47,879 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-10-21 10:49:47,879 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-10-21 10:49:48,208 sagemaker-containers INFO     Module main_train_bert does not provide a setup.py. 
Generating setup.py
2019-10-21 10:49:48,208 sagemaker-containers INFO     Generating setup.cfg
2019-10-21 10:49:48,209 sagemaker-

2019-10-21 10:49:57,746 - pytorch_pretrained_bert.modeling - INFO - Weights of BertForSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
2019-10-21 10:49:57,746 - pytorch_pretrained_bert.modeling - INFO - Weights from pretrained model not used in BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
     


2019-10-21 10:49:46 Training - Training image download completed. Training in progress.2019-10-21 10:54:18,305 - algorithms.BertTrain - INFO - Train set result details:
2019-10-21 10:54:18,345 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_18044182-b24f-435a-8e25-36fa7212e37c_20191021_105418.csv: 
[[4258    0]
 [ 884    0]]
2019-10-21 10:54:18,353 - algorithms.BertTrain - INFO - Train set result details: 0.0
2019-10-21 10:54:18,353 - algorithms.BertTrain - INFO - Validation set result details:
2019-10-21 10:54:26,199 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_03d5c9cf-f712-4a9a-9406-7577b9e473c4_20191021_105426.csv: 
[[576   0]
 [116   0]]
2019-10-21 10:54:26,201 - algorithms.BertTrain - INFO - Validation set result details: 0.0 
2019-10-21 10:54:26,201 - algorithms.BertTrain - INFO - Snapshotting because the current score 0.0 is greater than None 
2019-10-21 10:

2019-10-21 11:24:34,444 - algorithms.BertTrain - INFO - Train set result details:
2019-10-21 11:24:34,452 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_2a730e76-e448-4085-b9d0-8edb1ecf3419_20191021_112434.csv: 
[[4258    0]
 [ 884    0]]
2019-10-21 11:24:34,458 - algorithms.BertTrain - INFO - Train set result details: 0.0
2019-10-21 11:24:34,458 - algorithms.BertTrain - INFO - Validation set result details:
2019-10-21 11:24:42,361 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_2c7b7900-03a6-480c-9067-32155de74a0b_20191021_112442.csv: 
[[576   0]
 [116   0]]
2019-10-21 11:24:42,363 - algorithms.BertTrain - INFO - Validation set result details: 0.0 
2019-10-21 11:24:42,363 - algorithms.BertTrain - INFO - Snapshotting because the current loss 39.626810789108276 is lower than 39.629714757204056 
2019-10-21 11:24:42,363 - algorithms.ModelSnapshotCallback - INFO - Snappsho

EndpointConnectionError: Could not connect to the endpoint URL: "https://api.sagemaker.us-east-2.amazonaws.com/"